# 0. Imports


In [1]:
import scipy
import scipy.io.wavfile
from scipy import signal
#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob

#import cmath
#import math

import librosa

from collections import defaultdict

import json

import copy

import sys

import pycochleagram
import pycochleagram.utils
import pycochleagram.erbfilter, pycochleagram.subband

import os

# 1. Pre-processing

## 1.1 Segment continuous sounds into smaller chunks

In [9]:
sample_rate, data = scipy.io.wavfile.read('tester.wav')
sample_rate, data.shape

(44100, (89376,))

In [7]:
# https://librosa.org/doc/latest/generated/librosa.display.waveshow.html : 
#import librosa.display
#librosa.display.waveshow(data.astype('float'))


In [4]:
resampled = data
#resampled = signal.resample(data, num=50000, t=None, axis=0, window=None)
number_overlap = 100
sample_freq, segment_times, STFT = signal.stft(resampled,noverlap=number_overlap) # all have the same size already!
# noverlap: the number of samples two neighbouring DFT have in common / reverse of hop size

hop_size = int(segment_times[1] - number_overlap)

#STFT = mel_filterbank(sample_rate, hop_size, segment_times, STFT)
# of course its the filtered STFT

/home/c/anaconda3/lib/python3.7/site-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 2, using nperseg = 2
  .format(nperseg, input_length))


ValueError: noverlap must be less than nperseg.

## 1.2 Normalization

In [14]:
data = (data - data.mean())/data.std()
data

array([0.00208697, 0.00208697, 0.00208697, ..., 0.0577503 , 0.04310206,
       0.04603171])

## 1.3 Fourier transform (FT)
The FT should already be included in the packages imported in the feature extraction section below.

# 2. Feature Extraction

## 2.1 Feature selection

## 2.2 Audio signals carry redundant and/or irrelevant information

## 2.3 Feature computation 

### 2.3.1 Constant Q transform (CQT)

In [99]:
# https://librosa.org/doc/latest/generated/librosa.cqt.html :
cqt = librosa.cqt(data) # y: audio time series. Multi-channel is supported.
# it produces complex numbered output
cqt.shape

array([[-3.65983156e+00+6.79972656e-02j,  9.15170383e-01+3.66273064e+00j,
         3.66660956e+00-1.63080181e+00j, ...,
         7.04456052e-01+1.09329462e+01j,  9.87816360e+00-3.52447419e+00j,
        -6.16920553e+00-8.11721933e+00j],
       [ 2.54945656e+00+5.39669399e-02j,  1.71161015e+00-2.11115332e+00j,
        -9.60309115e-01-2.94217262e+00j, ...,
        -2.02370624e+00+5.68719245e-01j, -3.28376275e-01+2.13315666e+00j,
         1.82255872e+00+1.19061541e+00j],
       [ 2.02691785e+00-6.37737855e-02j,  3.75891283e-01-2.28048411e+00j,
        -2.48404047e+00-1.48695467e+00j, ...,
        -1.79115028e+00+6.80501562e+00j,  3.33635497e+00+7.05861442e+00j,
         7.68117462e+00+3.14362034e+00j],
       ...,
       [-6.53483192e-03+3.98480162e-05j, -1.24696220e-02+3.12765863e-03j,
        -2.38174066e-01-3.02976111e-01j, ...,
        -1.10051988e-01+7.63132561e-02j, -4.22451370e-02+1.34174212e-02j,
         1.17106218e-01+5.69362132e-02j],
       [-2.22692983e-03+1.10279701e-04j,  8.

### 2.3.2 Mel-frequency Cepstral Coefficient (MFCC)

In [100]:
# https://librosa.org/doc/latest/generated/librosa.feature.mfcc.html :
mfcc = librosa.feature.mfcc(data) # y: audio time series. Multi-channel is supported.
mfcc.shape

array([[-151.10595029,  -13.1204574 ,   66.97937561, ...,   59.36023239,
          58.12763292,   39.38341316],
       [  86.04548271,  165.86102251,  188.55969025, ...,  199.36629245,
         189.33434061,  172.48097178],
       [ -37.72146522,  -27.48396419,  -45.62408759, ...,  -58.14896861,
         -64.18582034,  -48.39701926],
       ...,
       [  -7.33296495,  -11.4583387 ,  -10.21777164, ...,    4.63169811,
          -1.48179792,   -1.40228823],
       [   0.63738928,   -6.75138996,   -8.15532307, ...,   -7.21083155,
          -7.65327216,   -4.22483511],
       [   6.55520787,    3.60593943,   -5.84548003, ...,   -1.12405118,
          -5.63415054,   -5.48980859]])

### 2.3.3 Cochleogram
https://pycochleagram.readthedocs.io/en/latest/
According to the github page:
This package contains four main modules:

- pycochleagram.erbfilter: Functions for generating ERB-cosine filters. These functions are available in the original MATLAB implementation.

- pycochleagram.subband: Functions for performing subband decomposition using filterbanks made with erbfilter. These functions are available in the MATLAB implementation.

- pycochleagram.cochleagram: Convenience methods for quickly generating cochleagrams. Also, provides functions for cochleagram inversion (i.e., generating a signal waveform from a cochleagram). These methods are not readily available in the MATLAB implementation (you would have to compose functions from pycochleagram.erbfilter and pycochleagram.subband). This is intended to help you get started.

- pycochleagram.utils: A collection of helpful methods for working with cochleagram generation, including some plotting and audio playback functions, as well as some fft-like methods that allow for easy switching between fftpack (numpy/scipy) and fftw. NOTE: when working with pyaudio and the audio playback functions in utils, the sound output can be very loud. Take caution when working with this method.

explanations: https://pycochleagram.readthedocs.io/en/latest/pycochleagram.html#module-pycochleagram.utils


In [37]:
# care about filters:
'''
pycochleagram.erbfilter.freq2erb(freq_hz)
pycochleagram.erbfilter.make_erb_cos_filters(signal_length, sr, n, low_lim, hi_lim, full_filter=False, strict=False)
pycochleagram.erbfilter.make_full_filter_set(filts, signal_length=None)
'''
# recommended for the upcoming cochlea creation:
filterbank, center_f, f = pycochleagram.erbfilter.make_erb_cos_filters_nx(signal_length=data.shape[0], sr=sample_rate, n=18, low_lim=1/10**6, hi_lim=sample_rate/2, sample_factor=1, padding_size=None, full_filter=True, strict=True)
# return 9.265 * np.log(1 + freq_hz / (24.7 * 9.265))
# n as number filters, normal: 20-40
# hi_lim=sample_rate/2 as the Nyquist limit
filterbank.shape

(22, 89376)

In [88]:
cochleagram = pycochleagram.subband.generate_subband_envelopes_fast(signal=data, filters=filterbank, padding_size=None, fft_mode='auto', debug_ret_all=False)
cochleagram.shape

(22, 89376)

### Store file as ARFF in ASCII format
How to save and define an ASCII file: https://fileinfo.com/extension/ascii <br>
How to create an ARFF file: https://www.geeksforgeeks.org/how-to-create-arff-file-in-weka-tool/ <br>
ARFF files are also ASCII files.

In [101]:
label = 0
# create the arff file which can be then used by weka:
with open('first_arff.arff', 'w') as f:
    f.write(('@relation firstset\n@attribute sample_id integer\n'))
    for i in range(cochleagram.flatten().shape[0]):
        f.write((f'@attribute feat{i} real\n'))
    f.write(('@attribute label integer\n\n@data\n'))
    
# write simple sample into file:
with open('first_arff.arff', 'a') as f:
    f.write(('1,'+','.join(cochleagram.flatten().astype(str).tolist()))+','+str(label)+'\n')

### Loop over all samples for Task 1 (weight classification)

In [3]:

# Plan: we need MFCC for each sample of same length, however, different duration length gives us different
# column number in STFT which represents time bins. Therefore, fix STFT according to the longest sample
# snippets:
max_duration = 0
max_signal = None

for file_ind, file in enumerate(glob.glob('wavs/*/*/*.wav')):
    y, sr = librosa.load(file)

    duration = librosa.get_duration(y=y, sr=sr)
    
    if duration > max_duration:
        max_duration = duration
        max_signal = y
        
# don't care about resampling as it makes the signal only smaller

sample_freq, segment_times, STFT_largest = signal.stft(max_signal,noverlap=0) 
# noverlap: the number of samples two neighbouring DFT have in common / reverse of hop size
# --> It follow by setting noverlap to 0 we get the longest STFT

feat_ar_col_number = STFT_largest.shape[1] 
# segment_times intervals of 256
'''
feat_ar_col_number = 522
max_signal_len = 133124
max_signal

'''

'\nfeat_ar_col_number = 522\nmax_signal_len = 133124\nmax_signal\n\n'

In [4]:
def one_hot_encode(number_labels, label_name):
    label_ar = np.zeros((number_labels,))
    label_dict = {'light':1, 'medium':2, 'heavy':3, 'left':1, 'right':2}
    label_ind = label_dict[label_name]
    return label_ind
    label_ar[label_ind] = 1
    return label_ar

In [34]:
# give info which features to extract:
wished_feat = input('Type in which kind of features you want to extract: m->mfcc, q->cqt, c->cochleagram, several at once, e.g. cxq') 
arff_name = 'feat_arff/heavy_right_cqt.arff'
##arff_name = 'mfcc_try_yesorno_upsampled_strict_E_sr.arff'
arff_name = 'feat_inputs/mfcc_c.arff'
feat_list = wished_feat.split('x')
feat_list.sort()

label_size_counter = np.zeros((2,))
label_direction_counter = 0

sample_rate = 44100
hop_val = 800#330
#sample_freq, segment_times, 
STFT_largest = librosa.stft(max_signal,hop_length=hop_val) 
feat_ar_col_number = STFT_largest.shape[1] 


for file_ind, file in enumerate(glob.glob('wavs/*/*/*.wav')):

    # find out labels and define an label array:
    file_name_parts = file.split('/')
    label_ele_size = file_name_parts[1]
    label_ele_direction = file_name_parts[2]
    file_name = ''.join(file_name_parts[-1].split('.')[0])
    
    label_ar_size = one_hot_encode(3, label_ele_size)
    label_ar_direction = one_hot_encode(2, label_ele_direction)
    
    # load the current wav file/sample:
    sample_rate, data = scipy.io.wavfile.read(file)
    # normalize it:
    data = (data - data.mean())/data.std()

    # extract features:
    
    for method_ind, method in enumerate(feat_list):
        
        if method=='c':
            
            diff_signal = max_signal_len - len(data)
            if diff_signal > 0:
                filler_signal = np.zeros((diff_signal,))
                data = np.concatenate((data, filler_signal), axis=0)
            #print('sig shape', signa)
            
            # recommended for the upcoming cochlea creation:
            filterbank, center_f, f = pycochleagram.erbfilter.make_erb_cos_filters_nx(signal_length=data.shape[0], sr=sample_rate, n=18, low_lim=1/10**6, hi_lim=sample_rate/2, sample_factor=1, padding_size=None, full_filter=True, strict=True)
            # return 9.265 * np.log(1 + freq_hz / (24.7 * 9.265))
            # n as number filters, normal: 20-40
            # hi_lim=sample_rate/2 as the Nyquist limit
        
            intermediate = pycochleagram.subband.generate_subband_envelopes_fast(signal=data, filters=filterbank, padding_size=None, fft_mode='auto', debug_ret_all=False)
            intermediate = intermediate.reshape((1,-1))
        elif method=='m':
            # https://librosa.org/doc/latest/generated/librosa.feature.mfcc.html :
            intermediate = librosa.feature.mfcc(data,n_mfcc=20,sr=sample_rate,hop_length=hop_val) # y: audio time series. Multi-channel is supported.  
            # default sr=22050, sample_rate from read out data: 44100

        elif method=='q':
            # https://librosa.org/doc/latest/generated/librosa.cqt.html :
            intermediate = abs(librosa.cqt(data,n_bins=20)) # y: audio time series. Multi-channel is supported.
            # it produces complex numbered output

        # merge features into one array:
        if not method=='c':
            diff_col = feat_ar_col_number - intermediate.shape[1]
            if diff_col > 0:
                filler = np.zeros((intermediate.shape[0],diff_col))
                intermediate = np.concatenate((intermediate,filler),axis=1)
            if diff_col < 0:
                intermediate = intermediate[:,:diff_col]

        # normalize values as they are inputted into methods:
        intermediate = (intermediate - intermediate.mean())/intermediate.std()

        if method_ind == 0:
            feat_ar = intermediate.flatten()
        else:
            feat_ar = np.concatenate((feat_ar, intermediate.flatten()), axis=1)


    loop_number = len(feat_ar.flatten())

    # create an empty arff file which is then filled with samples when not existing so far:
    if not os.path.exists(arff_name):
        
        with open(arff_name, 'w') as f:
            
            f.write(('@relation firstset\n'))
            
            for i in range(loop_number):
                
                f.write((f'@attribute feat{i} real\n'))
                
            f.write(('@attribute label_size {1,2,3}\n@attribute label_direction {1,2}\n\n@data'))
            
            #f.write(('@attribute label_size_light {yes,no}\n@attribute label_size_medium {yes,no}\n@attribute label_size_heavy {yes,no}\n@attribute label_direction_left {yes,no}\n@attribute label_direction_right {yes,no}\n\n@data'))

    # fill the arff file 
    # write simple sample into file:  
    #label_ar_size = label_ar_size.astype(int).astype(str)
    #label_ar_size[np.where(label_ar_size=='1')] = 'yes'
    #label_ar_size[np.where(label_ar_size=='0')] = 'no'

    #label_ar_direction = label_ar_direction.astype(int).astype(str)
    #label_ar_direction[np.where(label_ar_direction=='1')] = 'yes'
   # label_ar_direction[np.where(label_ar_direction=='0')] = 'no'
    
    with open(arff_name, 'a') as g:
        
        #g.write((f'\n0,'+','.join(feat_ar.astype(float).astype(str).tolist()))+','+','.join(label_ar_size.astype(int).astype(str).tolist())+','+','.join(label_ar_direction.astype(int).astype(str).tolist()))
        #if label_ar_size==3 and label_ar_direction==2:  
        g.write((f'\n'+','.join(feat_ar.astype(float).astype(str).tolist()))+','+str(label_ar_size)+','+str(label_ar_direction))
         
        # upsample every second sample which comes from a minority group:
        if label_ar_size==1 and label_ar_direction==2:

            if True: #label_size_counter[0]%2==0: # True:
                g.write((f'\n'+','.join(feat_ar.astype(float).astype(str).tolist()))+','+str(label_ar_size)+','+str(label_ar_direction))

            label_size_counter[0] += 1

            label_direction_counter += 1
                
        elif label_ar_size==3 and label_ar_direction==2:
              
            if True: #label_size_counter[1]%2==0: # True
                g.write((f'\n'+','.join(feat_ar.astype(float).astype(str).tolist()))+','+str(label_ar_size)+','+str(label_ar_direction))

            label_size_counter[1] += 1
            
            label_direction_counter += 1
        
print('done')

Type in which kind of features you want to extract: m->mfcc, q->cqt, c->cochleagram, several at once, e.g. cxqm
done


From WEKA follows the class distribution:<br>
light 557<br>
medium 1466<br>
heavy 618<br>
<br>
left 1707<br>
right 934<br>

# 3. Classification
https://stackoverflow.com/questions/19950889/predicting-a-numerical-value-using-knn-in-weka

## 3.1 estimate boundaries, distributions, class-membership

## 3.2 classify new data based on estimates